# Compute the spectrum of Fe X

In [1]:
using JAC

Multiply charged ions have attracted continuous interest due to recent observations by the Solar and heliospheric Observatory (SOHO), the Far Ultraviolet Spectroscopic Explorer (FUSE) and several other astrophysical missions. In the extreme ultra-violet (EUV) range of spectra, for instance, many of the spectroscopically observed lines belong to the iron-group elements owing to their high abundance in different objects. Until the present, however, several of these lines remained unclassified because reliable theoretical predictions are often not available that consistently include relativistic and correlation effects. 

To demonstrate the use of the JAC toolbox, we here consider the level structure of Fe$^{9+}$ ions which, in spectroscopic notation, give rise to the spectrum Fe X. Several medium-to-large scale computations have been carried out to calculate the excitation energies and transition probabilities of (and among) the levels from the [Ne] $3s^2 3p^5$ -> $3s^2 3p^4 3d + 3s 3p^6$ configuration. These three configurations comprise in total 31 fine-structure levels with total angular momenta $J = 1/2, ..., 9/2$ (cf. Dong et al., 1999). Although the $3s^2 3p^4 3d + 3s 3p^6$ levels mainly decay by electric-dipole (E1) transitions, also M1, E2 and M2 multipole transitions have been considered in the literature. Below, we show how this *complete* Fe X spectrum can be calculated approximately by just a single call within the JAC toolbox.



However, before we shall perform this computation, let us define a standard grid for the computations:

In [2]:
grid = Radial.Grid(true)
setDefaults("standard grid", grid)

(Re-) Define the standard grid with 392 grid points.


Moreover, we can show here how to select proper **physical units**, in which all the energies, rates, etc. are handled at input and output time. To see the present `settings` of this units, we can simply call:

In [3]:
Basics.display("settings")

Current settings of the JAC module:  
-----------------------------------  

  + Framework:                              relativistic
  + Energy unit:                            eV
  + Rate and transition probability unit:   1/s
  + Cross section unit:                     barn
  + Time unit:                              sec

  + A standard grid has been defined; cf. Defaults.getDefaults()



Of course, we could easily change any of these units by a proper call to `setDefaults()`, but we shall leave the present (standard) choice for the given computations. Call `setDefaults()` to change these settings:


In [4]:
? setDefaults

search: setDefaults



`Defaults.setDefaults()`       ... (re-) defines some 'standard' settings which are common to all the computations with the JAC module, and which can          be 'overwritten' by the user. –- An improper setting of some variable may lead to an error message, if recognized         immediately. The following defaults apply if not specified otherwise by the user: the framework is 'relativistic',          energies are given in eV and cross sections in barn. Note that, internally, atomic units are used throughout for          all the computations within the program. nothing is returned if not indicated otherwise.

## + `("framework: relativistic")`  or  `("framework: non-relativistic")`

## ... to define a relativistic or non-relativistic framework for all subsequent computations.

  * `("method: continuum, spherical Bessel")`  or  `("method: continuum, pure sine")`  or   `("method: continuum, asymptotic Coulomb")`  or  `("method: continuum, nonrelativistic Coulomb")`  or   `("method: continuum, Galerkin")`     ... to define a a method for the generation of the continuum orbitals as (pure) spherical Bessel, pure sine,       asymptotic Coulomb, nonrelativistic Coulomb orbital or by means of the B-spline-Galerkin method.
  * `("method: normalization, pure sine")`  or  `("method: normalization, pure Coulomb")`  or  `("method: normalization, Ong-Russek")`      ... to define a method for the normalization of the continuum orbitals as asymptotically (pure) sine or Coulomb        functions, or following the procedure by Ong & Russek (1978).
  * `("QED model: Petersburg")`  or  `("QED model: Sydney")`      ... to define a model for the computation of the QED corrections following the work by Shabaev et al. (2011; Petersburg)        or Flambaum and Ginges (2004; Syney).
  * `("unit: energy", "eV")`  or  `("unit: energy", "Kayser")`  or  `("unit: energy", "Hartree")`  or   `("unit: energy", "Hz")`  or  `("unit: energy", "Hz")`     ... to (pre-) define the energy units for all further printouts and communications with the JAC module.
  * `("unit: cross section", "a.u.")`  or  `("unit: cross section", "barn")`  or  `("unit: cross section", "Mbarn")`     ... to (pre-) define the unit for the printout of cross sections.
  * `("unit: rate", "a.u.")`  or  `("unit: rate", "1/s")`  ... to (pre-) define the unit for the printout of rates.
  * `("unit: resonance strength", "a.u.")`  or  `("unit: resonance strength", "barn eV")`  or   `("unit: resonance strength", "cm^2 eV")`  ... to (pre-) define the unit for the printout of resonance strengths.
  * `("unit: time", "a.u.")`  or  `("unit: time", "sec")`  or  `("unit: time", "fs")`  or  `("unit: time", "as")`     ... to (pre-) define the unit for the printout and communications of times with the JAC module.

---

  * `("relativistic subshell list", subshells::Array{Subshell,1}; printout::Bool=true)`     ... to (pre-) define internally the standard relativistic subshell list on which the standard order of orbitals is based.

---

  * `("standard grid", grid::Radial.Grid; printout::Bool=true)`     ... to (pre-) define internally the standard radial grid which is used to represent most orbitals.

---

  * `("QED: damped-hydrogenic", Znuc::Float64, wa::Array{Float64,1})`     ... to (re-) define the lambda-C damped overlap integrals of the lowest kappa-orbitals        [ wa*1s*1/2, wa*2p*1/2, wa*2p*3/2, wa*3d*3/2, wa*3d*5/2 ] for the (new) nuclear charge Znuc;        nothing is returned.



Apparently, we can call `JAC.define()` also in order to open some ASCII (.txt) file into which all the final (important) results are printed for later reference and inspection.


In [5]:
setDefaults("print summary: open", "09-output-radiative.txt")


Next, we shall now describe the computation to be carried out. Apart from a title line (String) and nuclear model (cf. tutorial 03-setting-the-nucleus.ipynb), we here need to specify the initial- and final-state configurations that are to be included into the computation. This is done by defining an instance of an:

```julia
comp = Atomic.Computation(Atomic.Computation(), name="Energies and Einstein coefficients for the spectrum Fe X", 
              grid=Radial.Grid(true), nuclearModel=Nuclear.Model(26.);              
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],              
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")],               
              process = Radiative,  processSettings = PhotoEmission.Settings(...) ); 
```

In principle, any number of configurations can be specified as long as they are listed explicitly. Here, we have selected also the identifier `Radiative` to consider the computation of transition probabilities between the levels of the three configurations given above. All further details about any computation can be specified by means of some proper `settings` that are specific to each particular process, here `PhotoEmission.Settings()`. To better understand to which extent we can affect and control these computations, let us look at the definition of this data struct:


In [6]:
? PhotoEmission.Settings

`struct  PhotoEmission.Settings  <:  AbstractProcessSettings`       ... defines a type for the details and parameters of computing radiative lines.

```
+ multipoles              ::Array{EmMultipoles}     ... Specifies the (radiat. field) multipoles to be included.
+ gauges                  ::Array{UseGauge}         ... Gauges to be included into the computations.
+ calcAnisotropy          ::Bool                    ... True, if the anisotropy (structure) functions are to be 
                                                        calculated and false otherwise 
+ printBefore             ::Bool                    ... True, if all energies and lines are printed before comput.
+ lineSelection           ::LineSelection           ... Specifies the selected levels, if any.
+ photonEnergyShift       ::Float64                 ... An overall energy shift for all photon energies.
+ mimimumPhotonEnergy     ::Float64                 ... minimum transition energy for which (photon) transitions 
                                                        are included into the computation.
+ maximumPhotonEnergy     ::Float64                 ... maximum transition energy for which (photon) transitions 
                                                        are included.
```

---

`PhotoEmission.Settings()`  ... constructor for the default values of radiative line computations

---

`PhotoEmission.Settings(set::PhotoEmission.Settings;`

```
    multipoles::=..,        gauges=..,          calcAnisotropy=..,          printBefore=..,
    line=..,                selectedLines=..,   photonEnergyShift=..,       mimimumPhotonEnergy=.., 
    maximumPhotonEnergy=..) 
                
... constructor for modifying the given PhotoEmission.Settings by 'overwriting' the previously selected parameters.
```


Of course, we can always apply the default values for any particular compuation which, for present case, are given by:

In [7]:
defaultsSettings = PhotoEmission.Settings()

multipoles:             EmMultipole[E1]  
gauges:                 UseGauge[UseCoulomb]  
calcAnisotropy:         false  
printBefore:            false  
lineSelection:          Inactive LineSelection.  
photonEnergyShift:      0.0  
mimimumPhotonEnergy:    0.0  
maximumPhotonEnergy:    10000.0  



Below, we wish to consider also other multipole transitions, namely E1, M1, E2, M2, and like perform all calculations within both, the Coulomb and Babushkin gauges. In addition, we shall **NOT** compute the (angular) anisotropy parameters nor do we wish to select individual transitions right from the beginning. This latter option might be interesting if large sets of configurations are to be considered. Moreover, we do not neither specify any (overall) shift nor restrict the photon energies, two features that are of particular interest for large data sets


We these remarks in mind, we now specify the atomic computation in the form:


In [8]:
photoSettings = PhotoEmission.Settings(defaultsSettings, multipoles=[E1, M1, E2, M2], gauges=[UseCoulomb, UseBabushkin],
                                       printBefore=true)

multipoles:             EmMultipole[E1, M1, E2, M2]  
gauges:                 UseGauge[UseCoulomb, UseBabushkin]  
calcAnisotropy:         false  
printBefore:            true  
lineSelection:          Inactive LineSelection.  
photonEnergyShift:      0.0  
mimimumPhotonEnergy:    0.0  
maximumPhotonEnergy:    10000.0  


In [12]:
comp = Atomic.Computation(Atomic.Computation(), name="Energies and Einstein coefficients for the spectrum Fe X",  
              grid=grid, nuclearModel=Nuclear.Model(26.);
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")], 
              processSettings = photoSettings ); 


All what is needed now is to **perform** this computation by the call:


In [13]:
perform(comp)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^1 3p_1/2^2 3p_3/2^4 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^0 3p_3/2^4 3d_3/2^0 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^0 3p_3/2^4 3d_3/2^1 3d_5/2^0 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^3 3d_3/2^0 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^3 3d_3/2^1 3d_5/2^0 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^2 3d_3/2^0 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^2 3d_3/2^1 3d_5/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.75657341

     19     20p_3/2      -1.61333010e-01    -8.45705558e-01    +4.24198711e+00    
     20     21p_3/2      -1.18223702e-01    -7.67051729e-01    +5.48813830e+00    
     21     22p_3/2      -8.92043971e-02    -6.98881076e-01    +6.83460344e+00    
     22     23p_3/2      -6.99467422e-02    -6.39410177e-01    +8.14138611e+00    
     23     24p_3/2      -5.73164532e-02    -5.87219333e-01    +9.24521407e+00    
     24     25p_3/2      -4.91642612e-02    -5.41167051e-01    +1.00073260e+01    
     25     26p_3/2      -4.42577673e-02    -5.00327101e-01    +1.03048428e+01    
     26     27p_3/2      +3.57610986e+01    -4.63941576e-01    +1.01297336e+00    
     27     28p_3/2      +1.41779094e+02    -4.31385479e-01    +1.00304266e+00    
     28     29p_3/2      +3.89440728e+02    -4.02139702e-01    +1.00103261e+00    
     29     30p_3/2      +9.23066139e+02    -3.75770208e-01    +1.00040709e+00    
     30     31p_3/2      +2.00929343e+03    -3.51911831e-01    +1.00017514e+00    
    


Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.5025822e+02;   self-cons'cy = 1.5361e-01  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.1323871e+01;   self-cons'cy = 4.6360e-01  [1.0000e+02 for sym-block kappa = -1]
  3s_1/2::  en [a.u.] = -8.6065408e+00;   self-cons'cy = 6.2987e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.8575070e+01;   self-cons'cy = 4.9888e-01  [1.0000e+02 for sym-block kappa = 1]
  3p_1/2::  en [a.u.] = -7.9198931e+00;   self-cons'cy = 6.5430e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -3.0200519e+01;   self-cons'cy = 4.7428e-01  [1.0000e+02 for sym-block kappa = -2]
  3p_3/2::  en [a.u.] = -8.3138150e+00;   self-cons'cy = 6.3839e-01  [1.0000e+02 for sym-block kappa = -2]
  3d_3/2::  en [a.u.] = -7.1702612e+00;   self-cons'cy = 6.8018e-01  [1.0000e+02 for sym-block kappa = 2]
  3d_5/2::  en [a.u.] = -7.2581389e+00;   self-cons'cy = 6.7635e-01  [1.0000e+02 for sym-block kappa = -3]

Iterat

Compute CI matrix of dimension 5 x 5 for the symmetry 7/2^+ ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 9/2^+ ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    1/2 +     -1.241474804820569e+03    -3.378225014807667e+04    -3.378225014807667e+04 
     2    7/2 +     -1.241013098372511e+03    -3.376968647569064e+04    -3.376968647569064e+04 
     3    5/2 +     -1.241013062920544e+03    -3.376968551099347e+04    -3.376968551099347e+04 
     4    3/2 +     -1.241007428134091e+03    -3.376953218064416e+04    -3.376953218064416e+04 
     5    1/2 +     -1.241000319075533e+03    -3.376933873330749e+04    -3.376933873330749e+04 
     6    9/2 +     -1.240875991688099e+03    -3.376595561277516e+04    -3.376595561277516e+04 
     7    7/2 +     -1.240851885178099e+03    -3.376529964122596e+04    -3.376529964122596e+04 
     8    1/2 +     -1.240842093483756e+03    -3.376503319565142e+04    -3.

Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -3.41095882e+02    -3.41097837e+02    +5.73311610e-06    
      2      2s_1/2      -8.54687357e+01    -8.54689584e+01    +2.60532193e-06    
      3      3s_1/2      -3.78995842e+01    -3.78994478e+01    -3.59853081e-06    
      4      4s_1/2      -2.12867978e+01    -2.12819385e+01    -2.28278309e-04    
      5      5s_1/2      -1.36512733e+01    -1.36039714e+01    -3.46501490e-03    
      6      6s_1/2      -1.02310357e+01    -9.43887322e+00    -7.74273999e-02    
      7      7s_1/2      -8.90147750e+00    -6.93005925e+00    -2.21470902e-01    
      8      8s_1/2      -6.53712432e+00    -5.30306679e+00    -1.88776819e-01    
      9      9s_1/2      -4.74910062e+00    -

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_3/2      -8.46909855e+01    -8.46909743e+01    -1.32105005e-07    
      2      3p_3/2      -3.76688688e+01    -3.76687634e+01    -2.79773042e-06    
      3      4p_3/2      -2.11887506e+01    -2.11846929e+01    -1.91502660e-04    
      4      5p_3/2      -1.36295960e+01    -1.35542226e+01    -5.53012776e-03    
      5      6p_3/2      -1.13122538e+01    -9.41010352e+00    -1.68149540e-01    
      6      7p_3/2      -9.13908230e+00    -6.91195226e+00    -2.43692962e-01    
      7      8p_3/2      -6.57908674e+00    -5.29094223e+00    -1.95793819e-01    
      8      9p_3/2      -4.77192752e+00    -4.17982022e+00    -1.24081369e-01    
      9     10p_3/2      -3.45174310e+00    -3.38519008e+00    -1.92809880e-02    
     10    

  2p_1/2::  en [a.u.] = -3.4016230e+01;   self-cons'cy = 6.8797e-07  [2.0050e-06 for sym-block kappa = 1]
  3p_1/2::  en [a.u.] = -9.1326197e+00;   self-cons'cy = 6.9509e-07  [2.0050e-06 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -3.3557603e+01;   self-cons'cy = 4.2355e-07  [4.2300e-06 for sym-block kappa = -2]
  3p_3/2::  en [a.u.] = -9.0554251e+00;   self-cons'cy = 5.1104e-07  [4.2300e-06 for sym-block kappa = -2]

Iteration 8 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.6454788e+02;   self-cons'cy = 5.3666e-08  [2.2401e-06 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.8106555e+01;   self-cons'cy = 1.9083e-07  [2.2401e-06 for sym-block kappa = -1]
  3s_1/2::  en [a.u.] = -1.0381243e+01;   self-cons'cy = 2.3170e-07  [2.2401e-06 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -3.4016229e+01;   self-cons'cy = 2.0880e-08  [8.0131e-08 for sym-block kappa = 1]
  3p_1/2::  en [a.u.] = -9.1326192e+00;   self-cons'cy = 2.7898e-08  [8.0131e-08 for sym-block kappa = 1]
  2p_3/2

Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [10-2] ... 
done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [10-2] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [11-1] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [11-1] ... 
done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [11-1] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [11-2] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [11-2] ... 
done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [11-2] ... 
done. 
Compute 

Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [28-1] ... 
done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [28-2] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [29-1] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [29-1] ... 
done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [29-1] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [29-2] ... 
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [29-2] ... 
done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [29-2] ... 
done. 
 
  Eins

        16 --    2      5/2 + --> 1/2 -     5.228055e+01        M2       Magnetic     1.755716e+01  4.562743e-09    4.441051e-10    1.155634e-14      
        17 --    1      5/2 + --> 3/2 -     5.466001e+01        E1       Coulomb      2.468044e+09  2.806118e-01    2.855586e-02    1.624496e-06      
        17 --    1      5/2 + --> 3/2 -     5.466001e+01        E1       Babushkin    2.371681e+09  2.696556e-01    2.744092e-02    1.561069e-06      
        17 --    1      5/2 + --> 3/2 -     5.466001e+01        M2       Magnetic     1.800154e+00  2.046741e-10    2.082822e-11    1.184883e-15      
        17 --    2      5/2 + --> 1/2 -     5.264475e+01        M2       Magnetic     1.576207e+02  4.011808e-08    3.932012e-09    1.037478e-13      
        19 --    1      7/2 + --> 3/2 -     5.555642e+01        M2       Magnetic     1.620193e+01  2.339186e-09    2.419461e-10    1.066430e-14      
        20 --    1      5/2 + --> 3/2 -     5.605251e+01        E1       Coulomb      3.641768


... and which shows all the intermediate output of the computations. First, all the initial-state levels are calculated *self-consistently* due to the default values for such SCF and configuration-interaction computations, and similarly for all final-state levels. These computations include the definition of the CSF basis, the generation of *start* orbitals as well as the computation of the angular coefficients, the SCF interation and up to the set-up and diagonalization of the corresponding Hamiltonian matrix. Obviously, this gives rise to a full representation of all atomic levels of interest.

These levels are then utilized to compute the (multipole) transitions amplitudes and rates. All final results are also tabulated in a neat format. However, since many of the internal tables of the JAC toolbox are *wider* than the standard widths of these notebooks, we shall first **close** our .txt summary file:


In [11]:
setDefaults("print summary: close", "")


This summary file **54-output-radiative.txt** can now be found in the current directory. As seen from these tables, the transition energies agree within 3..4 digits with a similar computation by other atomic structure codes, such as GRASP. Apart from the level splitting among the low-lying $3s 3p^6$, $3s^2 3p^4 3d$  $J=1/2$ levels, further comparision can be made also with the evaluated data from the NIST Spectroscopic Database, for instance, for the excitation energies from the $3s^2 3p^5$  $J = 3/2$ ground level.